In [1]:
import json
import os
import time
from pandas import json_normalize
import psycopg2
from psycopg2 import sql

import pandas as pd
import requests
from dotenv import load_dotenv

load_dotenv()

access_token = os.getenv("ACCESS_TOKEN")
HOST = os.getenv("HOST")
POSTGRES_DB = os.getenv("POSTGRES_DB")
POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")


# Informações de conexão com o banco de dados PostgreSQL
db_config = {
    "host": "localhost",
    "database": POSTGRES_DB,
    "user": POSTGRES_USER,
    "password": POSTGRES_PASSWORD,
}

# Registra o tempo antes da execução
start_prog = time.time()

In [2]:
print(access_token)

APP_USR-6628000663056293-110907-086ea536ccb8a8b229c3deda3af17e6f-233632476


## Verificando dados da conta

In [ ]:
url = "https://api.mercadolibre.com/users/me"

payload = {}
headers = {
  'Authorization': f'Bearer {access_token}'
}

response = requests.request("GET", url, headers=headers, data=payload)
response = response.text

print(response)

# Encontre posição do início e fim do campo "id"
start_pos = response.find('"id"')  # Encontra onde começa "id"
end_pos = response.find(',', start_pos)  # Encontra onde termina "id"

# Extraia o valor de "id"
seller_id = response[start_pos:end_pos].split(":")[1]
seller_id = seller_id.strip().strip('"')

print(f'O valor de "seller_id" é: {seller_id}')

## Carregando Inventory Ids de Produtos a serem pesquisados

Esses dados podem ser encontrados n coluna 'Código ML' da planilha 'Relação Full x Tiny' em 'Envios Full.xlsx'

In [ ]:
# Especifica caminho para o arquivo Excel
caminho_arquivo_excel = r'../Data/Base/Envios Full.xlsx'

# Especifica nome da planilha
nome_planilha = 'Relação Full x Tiny'

# Especifica nome da coluna que deseja carregar
coluna1 = 'Código ML' 
coluna2 = 'ID do anúncio'
coluna3 = 'ID Tiny'

# Df com colunas específicas
df_ml_estoque_id = pd.read_excel(caminho_arquivo_excel, sheet_name=nome_planilha, usecols=[coluna1,coluna2,coluna3])

# Adiciona prefixo "MLB" a cada linha na coluna especificada
# df_ml_id['ID do anúncio'] = 'MLB' + df_ml_id['ID do anúncio'].astype(str)

# Tamanho do df
df_ml_estoque_id.shape


In [ ]:
# Crie um DataFrame com valores não nulos em 'Código ML'
df_ml_id_no_null = df_ml_estoque_id[~pd.isnull(df_ml_estoque_id['Código ML'])]
df_ml_id_no_null.to_csv('../Data/Output/df_ml_id_no_null.csv', index=False)

# Crie um DataFrame com valores nulos em 'Código ML'
df_ml_id_with_null = df_ml_estoque_id[pd.isnull(df_ml_estoque_id['Código ML'])]
df_ml_id_with_null.to_csv('../Data/Output/df_ml_id_with_null.csv', index=False)

In [ ]:
# Verifica se há duplicatas
duplicates = df_ml_id_no_null.duplicated()
n_dup = duplicates.sum()

print(f'Número de duplicatas: {n_dup}')

# Encontre as duplicatas no DataFrame
duplicates = df_ml_id_no_null[df_ml_id_no_null.duplicated(keep=False)]

# Exiba as próprias duplicatas
duplicates

In [ ]:
# Remove as duplicatas e atualiza o DataFrame
df_ml_id_no_dup = df_ml_id_no_null.drop_duplicates()

df_ml_id_no_dup.shape

In [ ]:
# Qtd de itens sem Inventory ID
df_ml_id_with_null.shape


In [ ]:
df_ml_id_no_dup.head(1)

In [ ]:
df_ml_ids = df_ml_id_no_dup.copy()

In [ ]:
df_ml_ids.columns = ['ml_inventory_id', 'ml_code', 'tiny_id']
df_ml_ids.columns

In [ ]:
df_ml_ids.head(1)

## Populando tabela tiny_ml_codes no DB

In [ ]:

# Ela contém a relação entre tiny_id e inventory_id do ML 
# e não permite pares de valores duplicados entre (ml_inventory_id, tiny_id)

conn = psycopg2.connect(**db_config)

cursor = conn.cursor()

for index, row in df_ml_ids.iterrows():
    insert_query = sql.SQL("INSERT INTO tiny_ml_codes (ml_inventory_id, ml_code, tiny_id) VALUES (%s, %s, %s)")
    cursor.execute(insert_query, (row['ml_inventory_id'], row['ml_code'], row['tiny_id']))

conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()
print('Dados inseridos com sucesso!')

## Pegando Estoque de FulFillment

In [3]:
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM tiny_ml_codes"

df_codes = pd.read_sql(sql_query, conn)

df_codes.head(1)


C:\Users\couti\AppData\Local\Temp\ipykernel_16420\1288080312.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_codes = pd.read_sql(sql_query, conn)


,tiny_id,ml_inventory_id,ml_code,created_at,updated_at
0,509517168,JFGN34621,924922735,2023-11-07 15:46:15.284630,None


In [4]:
df_codes.shape

(754, 5)

In [5]:
df_codes['ml_inventory_id'].value_counts()

ml_inventory_id
ITDN80392    5
FSNB76403    5
DMTU81740    5
EIPZ51427    5
BXUC56742    5
            ..
GLMK71067    1
VFWO70372    1
EDGX70669    1
KYGE71098    1
UCQQ19395    1
Name: count, Length: 596, dtype: int64

In [6]:
codes = df_codes['ml_inventory_id'].unique()
print(len(codes))

596


In [7]:
codes

array(['JFGN34621', 'ERIM51807', 'HVCS52763', 'ZQIV51979', 'RBUB53192',
       'FXMN50419', 'TDFV51283', 'PKNN52962', 'MAEG54107', 'RYWV54124',
       'VIKN53907', 'MNSV53911', 'IXGP53982', 'XPLS45081', 'UWGO53377',
       'KVWJ40778', 'KBLG86592', 'APFU85474', 'CCQW73335', 'DXJZ75064',
       'JDMV76187', 'RJUL81287', 'TNXB83665', 'CIKE94068', 'HGUJ21007',
       'ZXFT21237', 'KBOK20943', 'MENA20785', 'CQAQ20361', 'HXJD21466',
       'GSNC20904', 'FRDZ21133', 'CNHG19808', 'DJDL21105', 'ABCB20467',
       'GLMK21133', 'VSPW20405', 'HTEZ21817', 'FVTQ20181', 'DCPP20168',
       'TXWS20511', 'XNOG20357', 'VSPW20414', 'PNST20989', 'EFTF21179',
       'ZZGN21450', 'KBOK20958', 'NQUG25995', 'WIZE26428', 'EDGX26018',
       'MHXR26588', 'YULB32421', 'XYHC31118', 'SJIT31691', 'ZZRV30730',
       'ITMG31457', 'NWXH31984', 'KRJP31426', 'BOWK32167', 'SDNY53354',
       'UQFO31564', 'SCKP53673', 'SCVF31712', 'MLVG32506', 'XDTG30724',
       'WGMG31517', 'PIJL35032', 'TSVR34841', 'HXJD35755', 'MOPJ

In [8]:
# pegando dados de fulfillment
counter = 0
json_list = []

# df_codes = df_codes.head(25)

for item in codes:
    url = f"https://api.mercadolibre.com/inventories/{item}/stock/fulfillment"

    payload = {}
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    print(f'Buscando dados {counter}/{len(codes)}: {item}')

    response = requests.get(url, headers=headers, data=payload)
    response_data = response.json()

    json_list.append(response_data)
    counter += 1
    
    if counter % 50 == 0:
        print(f"Fazendo uma pausa de 1 minuto...")
        time.sleep(60)

Buscando dados de: JFGN34621
Loop nº: 0
Buscando dados de: ERIM51807
Loop nº: 1
Buscando dados de: HVCS52763
Loop nº: 2
Buscando dados de: ZQIV51979
Loop nº: 3
Buscando dados de: RBUB53192
Loop nº: 4
Buscando dados de: FXMN50419
Loop nº: 5
Buscando dados de: TDFV51283
Loop nº: 6
Buscando dados de: PKNN52962
Loop nº: 7
Buscando dados de: MAEG54107
Loop nº: 8
Buscando dados de: RYWV54124
Loop nº: 9
Buscando dados de: VIKN53907
Loop nº: 10
Buscando dados de: MNSV53911
Loop nº: 11
Buscando dados de: IXGP53982
Loop nº: 12
Buscando dados de: XPLS45081
Loop nº: 13
Buscando dados de: UWGO53377
Loop nº: 14
Buscando dados de: KVWJ40778
Loop nº: 15
Buscando dados de: KBLG86592
Loop nº: 16
Buscando dados de: APFU85474
Loop nº: 17
Buscando dados de: CCQW73335
Loop nº: 18
Buscando dados de: DXJZ75064
Loop nº: 19
Buscando dados de: JDMV76187
Loop nº: 20
Buscando dados de: RJUL81287
Loop nº: 21
Buscando dados de: TNXB83665
Loop nº: 22
Buscando dados de: CIKE94068
Loop nº: 23
Buscando dados de: HGUJ210

In [9]:
len(json_list)

596

In [10]:
df = pd.DataFrame(json_list)
df

,inventory_id,total,available_quantity,not_available_quantity,not_available_detail,external_references
0,JFGN34621,71,71,0,[],"[{'type': 'item', 'id': 'MLB924922735'}]"
1,ERIM51807,36,34,2,"[{'status': 'transfer', 'quantity': 2}]","[{'type': 'item', 'id': 'MLB949788598'}]"
2,HVCS52763,3,1,2,"[{'status': 'transfer', 'quantity': 2}]","[{'type': 'item', 'id': 'MLB950214971'}]"
3,ZQIV51979,0,0,0,[],"[{'type': 'item', 'id': 'MLB950295475'}]"
4,RBUB53192,18,15,3,"[{'status': 'transfer', 'quantity': 3}]","[{'type': 'item', 'id': 'MLB950295955'}]"
...,...,...,...,...,...,...
591,RKDM61361,13,13,0,[],"[{'type': 'item', 'id': 'MLB2986538160', 'vari..."
592,PPWL59058,9,9,0,[],"[{'type': 'item', 'id': 'MLB2986538160', 'vari..."
593,RXXA58942,141,141,0,[],"[{'type': 'item', 'id': 'MLB2986538160', 'vari..."
594,POTW78210,2,2,0,[],"[{'type': 'item', 'id': 'MLB3106901850'}]"


In [11]:
df_full = df[['inventory_id', 'available_quantity']]
df_full

,inventory_id,available_quantity
0,JFGN34621,71
1,ERIM51807,34
2,HVCS52763,1
3,ZQIV51979,0
4,RBUB53192,15
...,...,...
591,RKDM61361,13
592,PPWL59058,9
593,RXXA58942,141
594,POTW78210,2


In [21]:
### AMBIENTE DE PRODUÇÃO ###
### Adicionando dados de estoque
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

for index, row in df_full.iterrows():
    insert_query = sql.SQL("""
        INSERT INTO fulfillment_stock (ml_inventory_id, available_quantity)
        VALUES (%s, %s)
    """)
    cursor.execute(insert_query, (row['inventory_id'], row['available_quantity']))

conn.commit()

cursor.close()
conn.close()

print('Dados inseridos com sucesso!')

Dados inseridos com sucesso!


In [22]:
df_full

,inventory_id,available_quantity
0,JFGN34621,71
1,ERIM51807,34
2,HVCS52763,1
3,ZQIV51979,0
4,RBUB53192,15
...,...,...
591,RKDM61361,13
592,PPWL59058,9
593,RXXA58942,141
594,POTW78210,2


In [23]:
### AMBIENTE DE TESTES ###
### Repetindo os dados com dias diferentes
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()
import datetime  

custom_created_at = datetime.datetime(2023, 11, 8, 1, 0, 0)  

for _ in range(30): # Repetir 30 vezes para adicionar dados com datas diferentes
    for index, row in df_full.iterrows(): # preenchendo db com dados de estoque
            insert_query = sql.SQL("""
                INSERT INTO fulfillment_stock (ml_inventory_id, available_quantity, created_at)
                VALUES (%s, %s, %s)
            """)
            cursor.execute(insert_query, (row['inventory_id'], row['available_quantity'], custom_created_at))
    custom_created_at += datetime.timedelta(days=-1)


conn.commit()

cursor.close()
conn.close()

print('Dados inseridos com sucesso!')

Dados inseridos com sucesso!


## É preciso o restante?

In [ ]:
df_er = json_normalize(json_list, record_path='external_references', meta=['inventory_id', 'total', 'available_quantity', 'not_available_quantity', 'not_available_detail'])
df_nad = json_normalize(json_list, record_path='not_available_detail', meta=['inventory_id', 'total', 'available_quantity', 'not_available_quantity', 'external_references'])


print('df',df.shape)
print('df_er',df_er.shape)
print('df_nad',df_nad.shape)

df_er.head(3)

In [ ]:
df_nad.head(3)

In [ ]:
print(df_nad.columns)
print(df_er.columns)

In [ ]:
df_nad_ = df_nad.drop(columns='external_references')
df_er_ = df_er.drop(columns='not_available_detail')

In [ ]:
common_cols = ['inventory_id', 'total', 'available_quantity', 'not_available_quantity']
# df_fulfillment = df_er_.merge(df_nad_, on=common_cols, how='left', suffixes=('_er','_nad'))
df_fulfillment = df_er_.merge(df_nad_, on=common_cols, how='left')

df_fulfillment.shape

In [ ]:
df_fulfillment.head(1)

In [ ]:
map_cols = {'inventory_id': 'ml_inventory_id', 'id': 'ml_item_id', 'status': 'nad_status','quantity':'nad_quantity'}
df_fulfillment = df_fulfillment.rename(columns=map_cols)
order_col = ['ml_inventory_id','ml_item_id','variation_id','nad_status','nad_quantity','total','available_quantity','not_available_quantity','type']
df_fulfillment = df_fulfillment[order_col]

df_fulfillment.head(1)

In [ ]:
df_fulfillment.shape

In [ ]:
df_fulfillment.info()

In [ ]:
df_fulfillment['variation_id'] = df_fulfillment['variation_id'].astype(str)
df_fulfillment['nad_quantity'] = df_fulfillment['nad_quantity'].fillna(0).astype(int)

In [ ]:
df_fulfillment.info()

In [ ]:
import psycopg2
from psycopg2 import sql

# Suponha que você já tenha seu DataFrame df_fulfillment

# Conecte-se ao banco de dados
conn = psycopg2.connect(**db_config)
cursor = conn.cursor()

# Itere pelas linhas do DataFrame e insira os dados na tabela
for index, row in df_fulfillment.iterrows():
    insert_query = sql.SQL("""
        INSERT INTO ml_fulfillment3 (ml_inventory_id, ml_item_id, variation_id, nad_status, nad_quantity, total, available_quantity, not_available_quantity, type)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """)
    cursor.execute(insert_query, (row['ml_inventory_id'], row['ml_item_id'], row['variation_id'], row['nad_status'], row['nad_quantity'], row['total'], row['available_quantity'], row['not_available_quantity'], row['type']))

# Confirme as alterações
conn.commit()

# Feche o cursor e a conexão
cursor.close()
conn.close()

print('Dados inseridos com sucesso!')


## Formatando

In [ ]:

# pegando codigos
conn = psycopg2.connect(**db_config)

sql_query = "SELECT * FROM tiny_ml_codes"

df_codes = pd.read_sql(sql_query, conn)

codes = df_codes['ml_inventory_id'].unique()

counter = 0
json_list = []

# diminuir codes

for item in codes:
    url = f"https://api.mercadolibre.com/inventories/{item}/stock/fulfillment"

    payload = {}
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    print(f'Buscando dados de: {item}')

    response = requests.get(url, headers=headers, data=payload)
    response_data = response.json()

    json_list.append(response_data)
    
    counter += 1
    
    if counter % 50 == 0:
        print(f"Fazendo uma pausa de 1 minuto...")
        time.sleep(60)
